<a href="https://colab.research.google.com/github/Miguel31Sa/Inteligencia-Artificial-MASS/blob/main/ExamenU2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Paso 1
!pip install tensorflow
!pip install pandas
!pip install matplotlib

# Paso 2: Importar las librerías
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

# Paso 3: Base de datos de las peliculas mas recomendadas
# Se sacara de la pagina movielens
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip ml-100k.zip

# Paso 4: Datos que se necesitaran para clasificar entre las mejores peliculas
ratings_df = pd.read_csv('ml-100k/u.data', sep='\t', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies_df = pd.read_csv('ml-100k/u.item', sep='|', header=None, encoding='latin-1', names=['movie_id', 'title'] + [f'genre_{i}' for i in range(1, 19)])

# Paso 5: Filtrar las películas de terror (poniendo como ejemplo que el usuario le gustan las peliculas de terror)
for i in range(1, 19):
    if f'genre_{i}' in movies_df.columns:
        movies_df[f'genre_{i}'] = movies_df[f'genre_{i}'].astype(str)

# Filtrar las películas de terror en cualquier columna de género
horror_movies = movies_df[movies_df[[f'genre_{i}' for i in range(1, 19)]].apply(lambda x: x.str.contains('Horror', na=False)).any(axis=1)]

# Comprobación de películas de terror
if horror_movies.empty:
    print("No se encontraron películas de terror en el conjunto de datos.")

    # En caso que no se encuentren las peliculas de terror escogera al azar diferentes peliculas para mostrarlas como recomendacion
    other_movies = movies_df.sample(n=3)
    print("Recomendaciones de películas de otros géneros:\n", other_movies[['movie_id', 'title']])
else:
    print("Películas de terror encontradas:\n", horror_movies[['movie_id', 'title']].head())

    # Paso 6: Calificaciones de las peliculas en donde se crea la matriz
    horror_ratings = ratings_df[ratings_df['movie_id'].isin(horror_movies['movie_id'])]
    horror_user_movie_matrix = horror_ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

    if horror_user_movie_matrix.shape[0] == 0:
        print("No hay calificaciones de películas de terror para crear la matriz de usuario-película.")

        other_movies = movies_df.sample(n=3)
        print("Recomendaciones de películas de otros géneros:\n", other_movies[['movie_id', 'title']])
    else:
        # Paso 7: Definir y entrenar el modelo
        X = horror_user_movie_matrix.values
        from sklearn.model_selection import train_test_split
        X_train, X_test = train_test_split(X, test_size=0.2, random_state=42)

        # Definir el modelo
        model = keras.Sequential([
            layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
            layers.Dense(64, activation='relu'),
            layers.Dense(X_train.shape[1], activation='sigmoid')
        ])

        # Compilar el modelo
        model.compile(optimizer='adam', loss='mean_squared_error')

        # Entrenar el modelo
        history = model.fit(X_train, X_train, epochs=50, batch_size=5, validation_data=(X_test, X_test))

        # Paso 8: Hacer recomendaciones de las peliculas
        predictions = model.predict(X)

        # Crear un DataFrame con las predicciones
        predicted_ratings_df = pd.DataFrame(predictions, columns=horror_user_movie_matrix.columns)

        # Obtener las recomendaciones para un usuario específico
        user_id = 1  # Cambia este valor para obtener recomendaciones de otros usuarios
        user_recommendations = predicted_ratings_df.iloc[user_id - 1].sort_values(ascending=False)

        # Filtrar las recomendaciones solo para películas de terror que no ha visto
        user_ratings = horror_user_movie_matrix.iloc[user_id - 1]
        unwatched_horror_movies = user_recommendations[user_ratings == 0]

        # Mostrar las mejores recomendaciones
        recommended_movies = unwatched_horror_movies.index.to_series().map(movies_df.set_index('movie_id')['title'])
        print("Recomendaciones para el usuario", user_id, "que no ha visto:")
        print(recommended_movies[recommended_movies.index.isin(unwatched_horror_movies.index)].sort_values(ascending=False).head(10))


--2024-10-28 03:18:28--  https://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip.5’

ml-100k.zip.5       100%[===================>]   4.70M  14.3MB/s    in 0.3s    

2024-10-28 03:18:28 (14.3 MB/s) - ‘ml-100k.zip.5’ saved [4924029/4924029]

Archive:  ml-100k.zip
replace ml-100k/allbut.pl? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml